In [ ]:
#!/usr/bin/env python3

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# 设置GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(42)
np.random.set_seed(42)

print('=' * 70)
print('改进版 LSTM 模型 - 德里气候温度预测')
print('=' * 70)

In [ ]:
def load_and_preprocess():
    train_path = '../data/DailyDelhiClimateTrain.csv'
    test_path = '../data/DailyDelhiClimateTest.csv'

    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)

    combined = pd.concat([train, test], ignore_index=True)
    combined['date'] = pd.to_datetime(combined['date'])
    combined = combined.sort_values('date').reset_index(drop=True)

    print(f'数据总天数: {len(combined)} 天')
    print(f'训练集: {len(train)} 天 ({len(train)/len(combined)*100:.1f}%)')
    print(f'测试集: {len(test)} 天 ({len(test)/len(combined)*100:.1f}%)')

    return combined, len(train)

def create_enhanced_sequences(df, window_size=14):
    base_features = ['humidity', 'meanpressure', 'wind_speed']

    df_features = df.copy()
    df_features['month'] = df_features['date'].dt.month
    df_features['day_of_year'] = df_features['date'].dt.dayofyear
    df_features['temp_lag1'] = df_features['meantemp'].shift(1)
    df_features['temp_lag7'] = df_features['meantemp'].shift(7)
    df_features['temp_rolling_mean_7'] = df_features['meantemp'].rolling(window=7).mean()
    df_features['humidity_rolling_mean_7'] = df_features['humidity'].rolling(window=7).mean()

    df_features = df_features.fillna(method='bfill').fillna(method='ffill')

    features = base_features + ['month', 'day_of_year', 'temp_lag1', 'temp_lag7']
    print(f'使用 {len(features)} 个特征: {features}')

    X, y = [], []
    for i in range(window_size, len(df_features)):
        X.append(df_features[features].iloc[i-window_size:i].values)
        y.append(df_features['meantemp'].iloc[i])

    return np.array(X), np.array(y), features

def build_improved_lstm(input_shape):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.LSTM(128, return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.LSTM(64, return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.LSTM(32, return_sequences=False,
                   kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu',
                    kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07
    )

    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model

In [ ]:
def visualize_results(y_true, y_pred, history, mse, rmse, mae, r2):
    os.makedirs('../results', exist_ok=True)

    fig, axes = plt.subplots(2, 3, figsize=(16, 10))

    axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch', fontsize=12)
    axes[0, 0].set_ylabel('Loss (MSE)', fontsize=12)
    axes[0, 0].set_title('Training History - Loss', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(history.history['mae'], label='Training MAE', linewidth=2)
    axes[0, 1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
    axes[0, 1].set_xlabel('Epoch', fontsize=12)
    axes[0, 1].set_ylabel('MAE (°C)', fontsize=12)
    axes[0, 1].set_title('Training History - MAE', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    axes[0, 2].plot(y_true[:100], label='True Temperature', linewidth=2, color='blue')
    axes[0, 2].plot(y_pred[:100], label='Predicted Temperature', linewidth=2, color='red')
    axes[0, 2].set_xlabel('Sample Index', fontsize=12)
    axes[0, 2].set_ylabel('Temperature (°C)', fontsize=12)
    axes[0, 2].set_title('Prediction Comparison (First 100 Samples)', fontsize=14, fontweight='bold')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    axes[1, 0].scatter(y_true, y_pred, alpha=0.5, s=10, color='green')
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    axes[1, 0].set_xlabel('True Temperature (°C)', fontsize=12)
    axes[1, 0].set_ylabel('Predicted Temperature (°C)', fontsize=12)
    axes[1, 0].set_title('True vs Predicted Values', fontsize=14, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    errors = y_true - y_pred
    axes[1, 1].hist(errors, bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
    axes[1, 1].set_xlabel('Prediction Error (°C)', fontsize=12)
    axes[1, 1].set_ylabel('Frequency', fontsize=12)
    axes[1, 1].set_title(f'Error Distribution (Mean={errors.mean():.2f}°C)', fontsize=14, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)

    axes[1, 2].scatter(y_pred, errors, alpha=0.5, s=10, color='purple')
    axes[1, 2].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[1, 2].set_xlabel('Predicted Temperature (°C)', fontsize=12)
    axes[1, 2].set_ylabel('Residuals (True - Predicted)', fontsize=12)
    axes[1, 2].set_title('Residual Plot', fontsize=14, fontweight='bold')
    axes[1, 2].grid(True, alpha=0.3)

    plt.suptitle(f'LSTM Model Performance: MSE={mse:.2f}, R²={r2:.3f}, RMSE={rmse:.2f}°C',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('../results/lstm_improved_results.png', dpi=150, bbox_inches='tight')
    plt.show()

    fig2, axes2 = plt.subplots(1, 2, figsize=(14, 5))
    axes2[0].plot(y_true, label='True Temperature', alpha=0.7, linewidth=1.5)
    axes2[0].plot(y_pred, label='Predicted Temperature', alpha=0.7, linewidth=1.5)
    axes2[0].fill_between(range(len(y_true)), y_true, y_pred,
                         where=(y_pred > y_true), color='red', alpha=0.3, label='Overestimation')
    axes2[0].fill_between(range(len(y_true)), y_true, y_pred,
                         where=(y_pred <= y_true), color='blue', alpha=0.3, label='Underestimation')
    axes2[0].set_xlabel('Test Sample Index', fontsize=12)
    axes2[0].set_ylabel('Temperature (°C)', fontsize=12)
    axes2[0].set_title('Full Test Set Prediction Comparison', fontsize=14, fontweight='bold')
    axes2[0].legend()
    axes2[0].grid(True, alpha=0.3)

    metrics = ['MSE', 'RMSE', 'MAE', 'R²']
    values = [mse, rmse, mae, r2]
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']
    bars = axes2[1].bar(metrics, values, color=colors, edgecolor='black')
    axes2[1].set_ylabel('Value', fontsize=12)
    axes2[1].set_title('Model Performance Metrics', fontsize=14, fontweight='bold')
    axes2[1].grid(True, axis='y', alpha=0.3)
    for bar, value in zip(bars, values):
        height = bar.get_height()
        if metrics[bars.index(bar)] == 'R²':
            axes2[1].text(bar.get_x() + bar.get_width()/2., height,
                         f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        else:
            axes2[1].text(bar.get_x() + bar.get_width()/2., height,
                         f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig('../results/lstm_performance_summary.png', dpi=150, bbox_inches='tight')
    plt.show()

    results_df = pd.DataFrame({
        'True_Temperature': y_true,
        'Predicted_Temperature': y_pred,
        'Error': errors
    })
    results_df.to_csv('../results/lstm_predictions.csv', index=False)
    print('预测结果已保存到: ../results/lstm_predictions.csv')

In [ ]:
def train_improved_model():
    print('[1/6] 正在加载数据...')
    combined, train_size = load_and_preprocess()

    print('[2/6] 正在创建时间序列...')
    window_size = 14
    X, y, features = create_enhanced_sequences(combined, window_size)

    X_train = X[:train_size - window_size]
    y_train = y[:train_size - window_size]
    X_test = X[train_size - window_size:]
    y_test = y[train_size - window_size:]

    print(f'训练集: {X_train.shape}')
    print(f'测试集: {X_test.shape}')

    print('[3/6] 正在归一化数据...')
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

    X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

    print('[4/6] 正在构建改进版 LSTM 模型...')
    input_shape = (window_size, len(features))
    model = build_improved_lstm(input_shape)
    model.summary()

    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=25,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            '../models/lstm_improved_best.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

    print('[5/6] 开始训练...')
    history = model.fit(
        X_train_scaled, y_train_scaled,
        batch_size=32,
        epochs=200,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )

    print('[6/6] 正在评估模型...')
    _test_loss, _test_mae, _test_mse = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)

    y_pred_scaled = model.predict(X_test_scaled, verbose=0).flatten()
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    y_test_original = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

    mse = mean_squared_error(y_test_original, y_pred)
    mae = mean_absolute_error(y_test_original, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, y_pred)

    print('[6/6] 测试集指标:')
    print(f'  MSE: {mse:.4f} (°C)²')
    print(f'  RMSE: {rmse:.4f} °C')
    print(f'  MAE: {mae:.4f} °C')
    print(f'  R²: {r2:.4f}')

    os.makedirs('../models', exist_ok=True)
    model.save('../models/lstm_improved_final.h5')
    print('模型已保存到: ../models/lstm_improved_final.h5')

    visualize_results(y_test_original, y_pred, history, mse, rmse, mae, r2)

    return mse, model, y_test_original, y_pred, history

In [ ]:
mse, model, y_true, y_pred, history = train_improved_model()
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_true - y_pred))
r2 = r2_score(y_true, y_pred)

print('训练结束，核心指标:')
print(f'  MSE: {mse:.4f} (°C)²')
print(f'  RMSE: {rmse:.4f} °C')
print(f'  MAE: {mae:.4f} °C')
print(f'  R²: {r2:.4f}')